# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
    return nx.read_edgelist('Employee_Movie_Choices.txt', delimiter="\t")

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    
    G = answer_one()
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type='employee')
        if node in movies:
            G.add_node(node, type='movie')
    
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
        
    G = answer_two()
    weighted_projected_graph = bipartite.weighted_projected_graph(G, employees)
    
    return weighted_projected_graph

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [5]:
# !cat 'Employee_Relationships.txt' 

In [6]:
def answer_four():
        
    G = answer_three()
    Rel = nx.read_edgelist('Employee_Relationships.txt' ,data=[('relationship_score', int)])

    G_df = nx.to_pandas_edgelist(G)
    G_df.columns = ['from', 'to', 'movie_score']

    # df = pd.DataFrame(G.edges(data=True), columns=['From', 'to', 'score'])
    Rel_df = nx.to_pandas_edgelist(Rel)
    Rel_df.columns = ['from', 'to', 'relationship_score']


    G_df2 = G_df.copy()
    G_df2.columns = ['to', 'from', 'movie_score']

    G_final_df = pd.concat([G_df, G_df2], sort=True)

    final_df = pd.merge(G_final_df, Rel_df, on = ['from', 'to'], how='right')
    final_df['movie_score'] = final_df['movie_score'].map(lambda x: 0. if np.isnan(x) else x)

    corr = final_df.corr().iloc[0,1]
    
    return corr

In [7]:
answer_four()

0.7883962221733474

In [8]:
# G = answer_three()
# print(G.edges(data=True), end='\n\n')

# Rel = nx.read_edgelist('Employee_Relationships.txt' ,data=[('relationship_score', int)])
# print(Rel.edges(data=True), end='\n\n')

# G_df = nx.to_pandas_edgelist(G)
# G_df.columns = ['from', 'to', 'movie_score']
# print(G_df, end='\n\n')

# # df = pd.DataFrame(G.edges(data=True), columns=['From', 'to', 'score'])
# Rel_df = nx.to_pandas_edgelist(Rel)
# Rel_df.columns = ['from', 'to', 'relationship_score']
# print(Rel_df, end='\n\n')

# G_df2 = G_df.copy()
# G_df2.columns = ['to', 'from', 'movie_score']

# G_final_df = pd.concat([G_df, G_df2], sort=True)

# final_df = pd.merge(G_final_df, Rel_df, on = ['from', 'to'], how='right')
# final_df['movie_score'] = final_df['movie_score'].map(lambda x: 0. if np.isnan(x) else x)
# print(final_df)

# final_df.corr()

[('Pablo', 'Frida', {'weight': 2}), ('Pablo', 'Andy', {'weight': 1}), ('Pablo', 'Vincent', {'weight': 1}), ('Frida', 'Andy', {'weight': 1}), ('Frida', 'Vincent', {'weight': 2}), ('Andy', 'Claude', {'weight': 1}), ('Andy', 'Lee', {'weight': 1}), ('Andy', 'Joan', {'weight': 1}), ('Andy', 'Georgia', {'weight': 1}), ('Claude', 'Georgia', {'weight': 3}), ('Lee', 'Joan', {'weight': 3})]

[('Andy', 'Claude', {'relationship_score': 0}), ('Andy', 'Frida', {'relationship_score': 20}), ('Andy', 'Georgia', {'relationship_score': -10}), ('Andy', 'Joan', {'relationship_score': 30}), ('Andy', 'Lee', {'relationship_score': -10}), ('Andy', 'Pablo', {'relationship_score': -10}), ('Andy', 'Vincent', {'relationship_score': 20}), ('Claude', 'Frida', {'relationship_score': 0}), ('Claude', 'Georgia', {'relationship_score': 90}), ('Claude', 'Joan', {'relationship_score': 0}), ('Claude', 'Lee', {'relationship_score': 0}), ('Claude', 'Pablo', {'relationship_score': 10}), ('Claude', 'Vincent', {'relationship_sco

,movie_score,relationship_score
movie_score,1.000000,0.788396
relationship_score,0.788396,1.000000
